# Implementation and linear cryptanalysis of a simplified AES-like cipher 
Laboratory session 1 of *Information Security*, AY 2024/25

To Do:
- precautions for input shape or type mismatches
- proper function documentation

In [ ]:
# library imports
import numpy as np
import sympy as sp

## Task 1
Using a programming language of your choice, implement the encryptor for a simplified
AES-like cipher.

Soooo let's do helper functions for 
- subkey generation
- substitution S
- transposition T
- linear transformation L

In [ ]:
def substitution(v):
    return (2*v) % 11

def transposition(y):
    # returns copy of y with 2nd half flipped
    # assuming y has an even length
    h = len(y)//2 # halfway point
    z = np.copy(y)
    z[h:]= z[h:][::-1]
    return z

A = np.array([[2,5],[1,7]])
def linear(z):
    W = z.reshape((2,4))
    w = (A @ W) % 11
    return w.flatten()

def subkeyGen(K):
    idx = np.array([
        [0, 2, 4, 6],
        [0, 1, 2, 3],
        [0, 3, 4, 7],
        [0, 3, 5, 6],
        [0, 2, 5, 7],
        [2, 3, 4, 5]
    ])
    return K[idx]

Put everything together in an encryptor function:

In [ ]:
def encryptA(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = (u + np.concatenate((subkeys[0], subkeys[0]))) % 11 

    for i in range(4):
        # S: perform substitution
        y = substitution(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = (w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11

    # last iteration without linear step:
    # S: perform substitution
    y = substitution(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = (z + np.concatenate((subkeys[5], subkeys[5])) ) % 11
        
    return v

Test with given test message & key:

In [ ]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptA(u,k)

## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field F = GF(p).

The transposition (flipping the 2nd half of the vector) is its own inverse here. For the linear transformation, 


In [ ]:
def substitutionInv(v):
    return 6*v % 11

A_inv = np.array([[2, 8],[6, 10]])
def linearInv(w):
    W = w.reshape((2,4))
    z = (A_inv @ W) % 11
    return z.flatten()

def decryptA(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    
    # subkey diff
    z = (x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11
    # T^-1: inverse transposition
    y = transposition(z)
    # S^-1: inv. substitution
    v = substitutionInv(y)

    for i in range(0, 4):
        # subkey diff
        w = (v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11
        # L^-1: inverse linear trafo
        z = linearInv(w)
        # T^-^: inverse transposition
        y = transposition(z)
        # S^-1: inv. substitution
        v = substitutionInv(y)

    # subkey diff
    u = (v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11
        
    return u

In [ ]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptA(encryptA(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the
previous slides and the substitution function described by the following table


In [ ]:
s_table = np.array([0, 2, 4, 8, 6, 10, 1, 3, 5, 7, 9])
s_table_inv = np.argsort(s_table)

def substitutionB(v):
    return s_table[v]

def substitutionInvB(v):
    return s_table_inv[v]
    
def encryptB(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionB(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionB(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)
        
    return v

def decryptB(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    
    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvB(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L 
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvB(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)
        
    return u

Test encryption with given message & key:

In [ ]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptB(u,k)

Test decryption:

In [ ]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptB(encryptB(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

## Task 6

In [ ]:
p = 11

def linear_matrices():
    A = []
    B = []
    z = np.zeros(8, dtype=int)
    for j in range(8):
        k = np.zeros(8, dtype=int); k[j] = 1
        A.append(encryptB(z, k))
    for j in range(8):
        u = np.zeros(8, dtype=int); u[j] = 1
        B.append(encryptB(u, z))
    A, B = np.array(A).T, np.array(B).T
    samples = [
        (np.array([10, 4, 3, 0, 2, 6, 9, 4]), np.array([5, 6, 1, 3, 0, 10, 0, 1])),
        (np.array([10, 2, 2, 3, 5, 9, 2, 3]), np.array([6, 10, 9, 0, 2, 9, 0, 5])),
        (np.array([0, 7, 0, 7, 4, 3, 1, 10]), np.array([7, 10, 7, 4, 4, 1, 8, 8])),
        (np.array([8, 4, 0, 8, 5, 5, 6, 5]), np.array([7, 0, 9, 6, 2, 1, 10, 6])),
        (np.array([9, 2, 0, 3, 7, 2, 2, 6]), np.array([9, 0, 5, 0, 10, 7, 4, 1]))
    ]
    return A, B, samples

def inv_matrix_mod(A_numpy, p):
    A_sym = sp.Matrix(A_numpy.tolist())
    if sp.gcd(A_sym.det(), p) != 1:
        raise ValueError("A not invertible mod p")
    return np.array(A_sym.inv_mod(p).tolist(), dtype=int)

# --- Task 6: Use linear approximation to recover key ---
A_lin, B, samples = linear_matrices() # A -> A_lin
u, x = samples[0]
C = np.eye(8, dtype=int)

Bu = (B @ u) % p
Cx = (C @ x) % p

# Compute inverse of A mod p
A_invB = inv_matrix_mod(A_lin, p) # A_inv -> A_invB

# Recover key
k = (A_invB @ (Cx - Bu)) % p

# Check approximation
x_check = (A_lin @ k + B @ u) % p
approx_vector = (A_lin @ k + B @ u + C @ x) % p
zero_count = np.count_nonzero(approx_vector == 0)
probability = zero_count / 8

# --- Output ---
print("\nRecovered key k:")
print(k.tolist())
print("\nCheck x:")
print("x (original):   ", x.tolist())
print("x (computed):   ", x_check.tolist())
print("\nLinear approximation check:")
print("Ak + Bu + Cx mod p =", approx_vector.tolist())
print(f"\nProbability of zero match: {probability:.2f}")

## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters ..

In [ ]:
def subkeyGenC(K):
    idx = np.array([
        [0, 1, 2, 3],
        [0, 1, 3, 2],
        [1, 2, 3, 0],
        [0, 3, 1, 2],
        [2, 3, 0, 1],
        [1, 3, 0, 2]
    ])
    return K[idx]

# modular multiplicative inverse table for GF(11)
# for v[j] = 0, we set v[j]^-1 to 0
inv_table = np.array([0, 1, 6, 4, 3, 9, 2, 8, 7, 5, 10])
def substitutionC(v):
    return 2*inv_table[v] % 11

def substitutionInvC(y):
    return inv_table[(6*y) % 11]
    
def encryptC(u, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionC(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionC(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)
        
    return v

def decryptC(x, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)
    
    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvC(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L 
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvC(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)
        
    return u

In [ ]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptC(u,k)

In [ ]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptC(encryptC(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

Task 8

In [ ]:
def key_gen():
    return np.random.randint(0, 11, 4)

def meet_in_the_middle(encrypt_fn, decrypt_fn, P_list, C_list, key_samples=80000):
    print("Generating random key candidates...")

    # Generate unique random keys for K1 and K2
    keys1 = np.unique([key_gen() for _ in range(key_samples)], axis=0).tolist()
    keys2 = np.unique([key_gen() for _ in range(key_samples)], axis=0).tolist()

    print(f"Keys1: {len(keys1)}, Keys2: {len(keys2)}")

    print("\n→ Computing encryptions of all plaintexts with each k1...")
    enc_map = {}
    for i, k1 in enumerate(keys1):
        all_outputs = []
        for P in P_list:
            out = encrypt_fn(np.array(P), np.pad(k1, (0, 4), constant_values=0))
            all_outputs.append(tuple(out.tolist()))
        enc_map[tuple(all_outputs)] = k1

    print("\n← Computing decryptions of all ciphertexts with each k2 and matching...")
    for j, k2 in enumerate(keys2):
        all_outputs = []
        for C in C_list:
            out = decrypt_fn(np.array(C), np.pad(k2, (0, 4), constant_values=0))
            all_outputs.append(tuple(out.tolist()))

        key = tuple(all_outputs)
        if key in enc_map:
            k1 = enc_map[key]
            print("\nFound matching key pair!")
            print("k1:", k1)
            print("k2:", k2)
            return k1, k2

    print("No match found.")
    return None, None

In [ ]:
plaintexts = [
    [4, 1, 6, 10, 2, 3, 5, 10],
    [10, 5, 4, 4, 7, 3, 2, 0],
    [2, 6, 8, 0, 6, 8, 10, 9],
    [3, 7, 2, 10, 1, 6, 9, 0],
    [5, 1, 6, 3, 10, 8, 8, 10],
]

ciphertexts = [
    [2, 1, 4, 0, 6, 7, 5, 5],
    [2, 8, 6, 2, 6, 2, 10, 0],
    [5, 5, 1, 4, 10, 2, 9, 2],
    [3, 8, 10, 10, 10, 9, 7, 8],
    [10, 0, 8, 10, 2, 10, 2, 2],
]

k1, k2 = meet_in_the_middle(encryptC, decryptC, plaintexts, ciphertexts, key_samples=80000)